## 32. Longest Valid Parentheses


Given a string containing just the characters `'('` and `')'`, find the length of the longest valid (well-formed) parentheses substring.

For `"(()"`, the longest valid parentheses substring is `"()"`, which has length = 2.

Another example is `")()())"`, where the longest valid parentheses substring is `"()()"`, which has length = 4.


DP问题：

定义 `dp(i)` 为 i 位置的最长括号结构的 长度（最优解）， 在最优解满足一下条件：

- `s(i) == '('` , `dp(i) = 0`
- `s(i) == ')'`
    - `s(i-1) == '('`,  `dp(i) = dp(i-2) + 2`
    - `s(i-1) == ')'`
        - `s(i-1-dp(i-1)) == '('`, `dp(i) = dp(i-1) + 2 + dp(i-2-dp(i-1))`
        - `s(i-1-dp(i-1)) == ')'`, `dp(i) = 0`
        



In [11]:
class Solution:
    def longestValidParentheses(self, s):
        """
        :type s: str
        :rtype: int
        """
        dp = [0] * len(s)
        maxlen = 0
        for i in range(1, len(s)):
            if s[i] == ')':
                if s[i-1] == '(':
                    dp[i] = dp[i-2] + 2
                elif i > dp[i-1] and s[i-1-dp[i-1]] == '(':
                    dp[i] = dp[i-1] + 2 + dp[i-2-dp[i-1]]
            print(i, dp[i], s[i], s[i-1-dp[i-1]])
            maxlen = max(maxlen, dp[i])
            
        return maxlen
    
# Solution().longestValidParentheses(')()())')
Solution().longestValidParentheses("(()))())(")

1 0 ( (
2 2 ) (
3 4 ) (
4 0 ) (
5 0 ( )
6 2 ) (
7 0 ) )
8 0 ( )


4

上面是典型的DP做法。还有更巧妙的办法。

其实这个括号结构有点特殊。仔细看会发现，它的模式就是：

```
expr = '()' |  '(' + expr + ')' | expr + expr
```

除去最后的 `expr + expr`，  `expr` 总是对称的。

一种想法是，从左向右scan一遍， 对 `(` 和 `)` 计数， 如果`)`的数量超过了 `(`的数量，说明不是有效结构，重置计数； 如果 `)` 的数量小于 `(` 的数量，则说明还有一些括号需要不全； 如果二者相等，则说明结构合理。

如果 `(` 的数量总是超过 `)`， 从右向左scan一遍，就可以解决问题。


In [13]:
class Solution:
    def longestValidParentheses(self, s):
        """
        :type s: str
        :rtype: int
        """
        maxlen = 0
        left = right = 0
        for c in s:
            if c == '(':
                left += 1
            else:
                right += 1
            if left == right:
                maxlen = max(maxlen, right * 2)
            elif right > left:
                left = right = 0
        left = right = 0
        for c in s[::-1]:
            if c == '(':
                left += 1
            else:
                right += 1
                
            if left == right:
                maxlen = max(maxlen, right * 2)
            elif left > right:
                left = right = 0
        return maxlen
# Solution().longestValidParentheses(')()())')
Solution().longestValidParentheses("(()))())(")

4

## 64. Minimum Path Sum

Given a m x n grid filled with non-negative numbers, find a path from top left to bottom right which minimizes the sum of all numbers along its path.

Note: You can only move either down or right at any point in time.

```
Example 1:
[[1,3,1],
 [1,5,1],
 [4,2,1]]
Given the above grid map, return 7. Because the path 1→3→1→1→1 minimizes the sum.

```

典型的DP问题。很简单， `dp[i, j] = min(dp[i-1, j], dp[i, j-1]) + gird[i, j]`

算法复杂度 $O(nm)$， 空间复杂度 $O(nm)$。 其实可以只用一行，简化空间复杂度。

In [19]:

class Solution:
    def minPathSum(self, grid):
        """
        :type grid: List[List[int]]
        :rtype: int
        """
        dp = []
        s = 0
        for x in grid[0]:
            s += x
            dp.append(s)
        # print(dp)    
        for i in range(1, len(grid)):
            dp[0] += grid[i][0]
            for j in range(1, len(dp)):
                dp[j] = min(dp[j-1], dp[j]) + grid[i][j]
            # print(dp)
        return dp[-1]
    
Solution().minPathSum([[1,3,1], [1, 5, 1], [4, 2, 1]])

7

## 72. Edit Distance

Given two words word1 and word2, find the minimum number of steps required to convert word1 to word2. (each operation is counted as 1 step.)

You have the following 3 operations permitted on a word:

a) Insert a character
b) Delete a character
c) Replace a character

题目大意是 两个字符串， 只允许3种操作，问最少需要多少操作可以将word1 转成word2？

也是动态规划问题：令 `dp[i,j]` 是问题 `word1[0:i]` 和 `word2[0:j]` 的解。则：

1. `dp[i][0] = i` ，一个空字符串和一个字符串，单纯用插入操作就可以解决。
2. `dp[0][j] = j` ，同上。
3. 如果`word1[i - 1] = word2[j - 1]`， 则 `dp[i][j] = dp[i - 1][j - 1]`
4. 否则 `dp[i][j] = min(dp[i - 1][j - 1] + 1, dp[i - 1][j] + 1, dp[i][j - 1] + 1)`
    1. 拆开看就是 替换、插入或者删除 三种情况的最优选择。

上面的过程可以对空间进行优化：
```
(i-1, j-1),  (i-1, j)
(i,   j-1),  (i,   j)
```
可以看出就是一个网格结构。

所以跟上个一样，维护一行就行了。 并且还可以选择只维护短的word所代表的行。


```
prev,   cache
dp[j],  dp[j+1]
```

In [31]:
class Solution:
    def minDistance(self, word1, word2):
        """
        :type word1: str
        :type word2: str
        :rtype: int
        """
        n, m = len(word1), len(word2)
        if m > n:
            n, m, word1, word2 = m, n, word2, word1
        if m == 0:
            return n
        
        dp = list(range(m+1))
        for i in range(n):
            prev = i
            dp[0] = i + 1
            for j in range(m):
                cache = dp[j+1]
                if word1[i] == word2[j]:
                    dp[j+1] = prev
                else:
                    dp[j+1] = min(prev, dp[j], cache) + 1
                prev = cache
                
        return dp[m]
    
# Solution().minDistance('abc', 'efg')
Solution().minDistance('a', 'a')
        

0

## 87. Scramble String

s1 和 s2 是两个字符串

如果 s1 的树状表示，在叶节点翻转后组成的字符串表示为 s2，则 s2 是 s1的 scramble。。。

也是DP问题。 关键是这个树状结构定义怎么定义。

思路非常简单， 一个 `s1, s2` 问题，可以划分成 `4*n` 个子问题： `s1[:i], s2[:i]`和 `s1[i:], s2[i:]`， 或者 `s1[:i], s2[-i:]` 和 `s1[-i:], s2[:i]`。 但是这样递归下去，感觉规模非常庞大。

考虑所有索引位置， 大约有 $n^4$ 个子问题。 但是大部分都无效，因为长度不一致。

In [56]:

class Solution:
    # @return a boolean
    def isScramble(self, s1, s2):
        n, m = len(s1), len(s2)
        if n != m or sorted(s1) != sorted(s2):
            return False
        if n < 4 or s1 == s2:
            return True
        f = self.isScramble
        for i in range(1, n):
            if f(s1[:i], s2[:i]) and f(s1[i:], s2[i:]) or \
               f(s1[:i], s2[-i:]) and f(s1[i:], s2[:-i]):
                return True
        return False
    
# Solution().isScramble("great", "rgeat")
Solution().isScramble("abcdefghijklmn", "efghijklmncadb")

False

In [58]:
class Solution:
    # @return a boolean
    def isScramble(self, s1, s2):
        n, m = len(s1), len(s2)
        if n != m:
            return False
        
        def dp(ss1, ss2):
            # 这是可以节省运行时间的关键：如果字符都不匹配，一定是错误的
            if sorted(ss1) != sorted(ss2):
                return False
            sn = len(ss1)
            if sn < 4 or ss1 == ss2:
                return True
            for i in range(1, sn):
                if (dp(ss1[:i], ss2[:i]) and dp(ss1[i:], ss2[i:])) or \
                       (dp(ss1[:i], ss2[-i:]) and dp(ss1[i:], ss2[:-i])):
                    return True
            return False        
        return dp(s1, s2)
    
# Solution().isScramble("abc", "bca")
Solution().isScramble("abcdefghijklmn", "efghijklmncadb")

False

## 95. Unique Binary Search Trees II

构建所有的二分查找树。对于 `1...n`

这也是个DP问题。

$$ DP(1, n) = \sum_i{DP(1, i-1) + DP(i+1, n)} $$

实际上也是一个DFS 搜索问题。

In [90]:
class Solution(object):
    def generateTrees(self, n):
        """
        :type n: int
        :rtype: List[TreeNode]
        """
        if n == 0:
            return []
        return self.dfs(1, n+1)
        
    def dfs(self, start, end):
        if start == end:
            return None
        result = []
        for i in range(start, end):
            for l in self.dfs(start, i) or [None]:
                for r in self.dfs(i+1, end) or [None]:
                    node = TreeNode(i)
                    node.left, node.right  = l, r
                    result.append(node)
        return result
    
    
def print_solution(n):
    def print_nodes(node):
        if node is None:
            print("null", end=',')
        else:
            print(node.val, end=',')
            print_nodes(node.left)
            print_nodes(node.right)
        
    list_trees = Solution().generateTrees(n)
    print(list_trees)
    for tree in list_trees:
        print_nodes(tree)
        print()

print_solution(3)

[<__main__.TreeNode object at 0x110ab86d8>, <__main__.TreeNode object at 0x110ab8cc0>, <__main__.TreeNode object at 0x110ab86a0>, <__main__.TreeNode object at 0x110ab84e0>, <__main__.TreeNode object at 0x110ab8390>]
1,null,2,null,3,null,null,
1,null,3,2,null,null,null,
2,1,null,null,3,null,null,
3,1,null,2,null,null,null,
3,2,1,null,null,null,null,


下面是 generator版本的

In [93]:
# Definition for a binary tree node.
class TreeNode:
    def __init__(self, x):
        self.val = x
        self.left = None
        self.right = None
    
    def __str__(self):
        return str(self.val)

class Solution(object):
    def generateTrees(self, n):
        """
        :type n: int
        :rtype: List[TreeNode]
        """
        if n == 0:
            return []
        
        def trees(i, j):
            if i > j:
                yield None
            for k in range(i, j+1):
                for left in trees(i, k-1):
                    for right in trees(k+1, j):
                        node = TreeNode(k)
                        node.left, node.right = left, right
                        yield node              
            
        return list(trees(1, n))
    
# Solution().generateTrees(3)
print_solution(3)

[<__main__.TreeNode object at 0x110b58780>, <__main__.TreeNode object at 0x110b58898>, <__main__.TreeNode object at 0x110b58940>, <__main__.TreeNode object at 0x110b589e8>, <__main__.TreeNode object at 0x110b58a90>]
1,null,2,null,3,null,null,
1,null,3,2,null,null,null,
2,1,null,null,3,null,null,
3,1,null,2,null,null,null,
3,2,1,null,null,null,null,


## 97. Interleaving String

交织string：给定 s1, s2, s3， 判断 s3 是否由 s1 和 s2 交织而成

例如：
```
Given:
s1 = "aabcc",
s2 = "dbbca",

When s3 = "aadbbcbcac", return true.
When s3 = "aadbbbaccc", return false.
```

一开始有一个简单的想法： 就是从 s3中划掉 s1的字符，剩下的如果和 s2相同，就是交织而成的。其实不对，有个反例：
```
s1="abc"
s2="bcd"
s3="abcbdc"
```

还是老老实实用 DP。

DP的思路其实也很简单：令 `DP(i, j)` 表示 `s1[i:]` 、`s2[j:]` 和 `s3[i+j:]`的解

- 如果 `s1[i] == s2[j] == s3[i+j]` ， 则 `DP(i, j) = DP(i+1, j) or DP(i, j+1)`
- 如果 `s1[i] != s2[j]`， 
    - 如果 `s3[i+j] == s1[i]`， 则 `DP(i, j) = DP(i+1, j)`
    - 如果 `s3[i+j] == s2[j]`， 则 `DP(i, j) = DP(i, j+1)`
    - 都不相同， 则 `DP(i, j) = False`
    
反向的思路：


DP的思路其实也很简单：令 `DP(i, j)` 表示 `s1[:i]` 、`s2[:j]` 和 `s3[:i+j]`的解

- 如果 `s1[i] == s2[j] == s3[i+j]` ， 则 `DP(i, j) = DP(i-1, j) or DP(i, j-1)`
- 如果 `s1[i] != s2[j]`， 
    - 如果 `s3[i+j] == s1[i]`， 则 `DP(i, j) = DP(i-1, j)`
    - 如果 `s3[i+j] == s2[j]`， 则 `DP(i, j) = DP(i, j-1)`
    - 都不相同， 则 `DP(i, j) = False`

- `DP(i, 0) = s1[:i] == s3[:i]`
- `DP(0, j) = s2[:j] == s3[:j]`

    
注意这也是一个网格样式的DP（正向和反向都一样）

```
x         (i-1, j)
(i, j-1)  (i, j)
```


这里注意一个大trick：  就是索引的问题： 注意： `a` 和 `b` 可以拼成 `ab`，那么 索引呢？ ， 0, 0，0，永远到不了1。 使用这种索引方式有很大缺陷的： `i in range(n), j in range(m)`，则永远无法判断 s3的最后一个字符。必须用正常索引方式： `i in range(1, n+1), j in range(1, m+1), k = i + j`，实际用在字符串索引时，都减1

In [125]:
class Solution:
    def isInterleave(self, s1, s2, s3):
        """
        :type s1: str
        :type s2: str
        :type s3: str
        :rtype: bool
        """
        n, m, l = len(s1), len(s2), len(s3)
        if n + m != l:
            return False
        if m > n:
            n, m, s1, s2 = m, n, s2, s1
            
        if m == 0:
            return s1 == s3
        
        dp = [False] * (m + 1)
        for i in range(n+1):
            for j in range(m+1):
                if (i == 0 and j == 0):
                    dp[j] = True
                elif (i == 0):
                    dp[j] = dp[j-1] and s2[j-1] == s3[j-1]
                elif (j == 0):
                    dp[j] = dp[j] and s1[i-1] == s3[i-1]
                else:
                    dp[j] = (dp[j] and s1[i-1] == s3[i+j-1]) or (dp[j-1] and s2[j-1] == s3[i+j-1])
        return dp[-1]
    
print(Solution().isInterleave('abc', 'bcd', 'abcbdc'))
print(Solution().isInterleave('aabcc', 'dbbca', 'aadbbcbcac'))
print(Solution().isInterleave('aabcc', 'dbbca', 'aadbbbaccc'))
print(Solution().isInterleave('', '', ''))
print(Solution().isInterleave('a', '', 'a'))
print(Solution().isInterleave('a', 'b', 'ab'))
                

True
True
False
True
True
True


## ☆☆115. Distinct Subsequences

Given a string S and a string T, count the number of distinct subsequences of S which equals T.

A subsequence of a string is a new string which is formed from the original string by deleting some (can be none) of the characters without disturbing the relative positions of the remaining characters. (ie, "ACE" is a subsequence of "ABCDE" while "AEC" is not).

Here is an example:
```
S = "rabbbit", T = "rabbit"

Return 3.
```

题目大意是 ：  问 S有多少个不同的 子串等于 T。 这里不同意思是截取的位置不同。

这里是解法：<https://leetcode.com/problems/distinct-subsequences/discuss/37327/Easy-to-understand-DP-in-Java>

不看真不知道，解释的很清楚。是个DP问题。

定义 `DP(i, j)` 是 问题 `S[:i]` 和 `T[:j]`的解。

- `T == ''`， `DP(i, 0) = 1`，因为空串是任意字符串的子串，且可以认为是唯一。
- `T != '', S = ''`, `DP(0, j) = 0`
- 首先看 `DP(i+1, j+1)`， 有一部分解 `DP(i+1, j+1)  = DP(i,j+1)`，即 `S[:i]`中包括 `T[:j+1]`的解，如果不考虑其他情况， 如果 `S[:i]` 有若干个 `T[:j+1]`的解。
- 如果 `S[i] == T[j]`， 则如果 `S[:i]` 有若干个 `T[:j]`子串， 则需要加上这一部分。 因此 `S[i] == T[j]`时，  `DP(i+1,j+1) = DP(i, j+1) + DP(i, j)`

这样看就很清楚：

```
  S 0123....i
T +----------+
0 |1111111111|
1 |0         |
2 |0         |
. |0         |
. |0         |
j |0         |
```
比如 `s = 'acdbabefbc', t = 'abc'`，则：

```
   s  =  acdbabefbc
'' dp = 11111111111     # 因为 ""空字符串是所有字符串的子串，只取1
'a'   = 01111222222     # 'a' 出现的时候，开始计数
'b'   = 00001           # 'b' 出现第一次，必须前面有 'a'才算第一次。
      = 0000113         # 'b' 出现第二次，感觉像个斐波那契问题：这个'b'和前面两个'a' 构成2个子串，在加上前一个'ab'，就是3个
      = 00001133355     # 'b' 出现第三次，这个'b' 和前面两个'a' 构成2个子串，再加上前面3个， 就是5个。
'c'   = 00_             # 'c' 出现第一次，因为前面没有'ab'，所以不计数
      = 00000000005     # 'c' 出现第二次，因为前面有5个'ab'子串，所以，总共有5个。
```

```
   s  =  rabbbbit
'' dp = 111111111     # 因为 ""空字符串是所有字符串的子串，只取1
'r'   = 011111111     # 'r' 出现的时候，开始计数
'a'   = 001111111     # 
'b'   = 000123444     # 
'b'   = 000013666     # 注意这里，计算的时候用来左上角，而不是正上方。
'i'   = 000000066
```

用行表示：

```
prev   cache
dp[i-1]  dp[i]
```


In [140]:
class Solution:
    def numDistinct(self, s, t):
        """
        :type s: str
        :type t: str
        :rtype: int
        """
        n, m = len(s), len(t)
        
        dp = [1] * (n+1)
        for j in range(m):
            prev = dp[0]
            dp[0] = 0
            for i in range(n):
                cache = dp[i+1]
                dp[i+1] = dp[i]
                if s[i] == t[j]:
                    dp[i+1] += prev
                prev = cache
            # print(dp)
        return dp[-1]

# Solution().numDistinct('acdbabefbc', 'abc')
Solution().numDistinct("rabbbit", "rabbit")

3

##  120. Triangle

Given a triangle, find the minimum path sum from top to bottom. Each step you may move to adjacent numbers on the row below.

For example, given the following triangle
```
[
     [2],
    [3,4],
   [6,5,7],
  [4,1,8,3]
]
The minimum path sum from top to bottom is 11 (i.e., 2 + 3 + 5 + 1 = 11).
```

简单的DP问题。

令 `DP(i, j)` 为第 `i`行，第`j`位置的最优解，有：

- `DP(i+1, j) = min(DP(i, j-1), DP(i, j)) + num(i+1, j)`

还可以反过来想，看看对不对：

- `DP(i, j) = min(DP(i+1, j), DP(i+1, j+1)) + num(i, j)`

居然是对的，哈哈

In [150]:

class Solution:
    def minimumTotal(self, triangle):
        """
        :type triangle: List[List[int]]
        :rtype: int
        """
        dp = triangle[-1]
        for i in range(len(triangle)-2, -1, -1):
            for j in range(i+1):
                dp[j] = min(dp[j], dp[j+1]) + triangle[i][j]
        
        return dp[0]
    
Solution().minimumTotal([[2],[3,4],[6,5,7],[4,1,8,3]])

11

## ☆ 53. Maximum Subarray

Find the contiguous subarray within an array (containing at least one number) which has the largest sum.

```
For example, given the array [-2,1,-3,4,-1,2,1,-5,4],
the contiguous subarray [4,-1,2,1] has the largest sum = 6.
```

典型的DP问题。但是最优解的子结构比较难找。

定义  `DP(i)` 是 `A[:i]` 问题的最优解，但是注意这里问题不再是找到子数组， 而是带有 `A[i-1]`的连续数值的子数组的和的最大值， `i in range(1, len(A)+1)`。  换句话说，对于 `A[:i]`，我只要找到 `A[i-1]`之前累积的最大和就可以了。

对于这样的问题，DP方法就很简单了：
- 如果 `DP(i) > 0`， `DP(i+1) = DP(i) + A[i]`， 如果前一段的和的最大值大于0， 那么继续累积好了。
- 否则： `DP(i+1) = A[i]`。 否则，就重新从 `A[i]`开始。


In [154]:
class Solution:
    def maxSubArray(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        maxsum = s = nums[0]
        for n in nums[1:]:
            if s > 0:
                s += n
            else:
                s = n
            maxsum = max(s, maxsum)
        return maxsum
    
Solution().maxSubArray([-2,1,-3,4,-1,2,1,-5,4])

6

## ☆☆152. Maximum Product Subarray

Find the contiguous subarray within an array (containing at least one number) which has the largest product.

```
For example, given the array [2,3,-2,4],
the contiguous subarray [2,3] has the largest product = 6.
```

DP问题的定义跟上面很相似。但是乘积比求和，要复杂一点。

令 `DP(i)` 是 `A[:i]`问题的最优解，以 `A[i-1]`结尾的连续子数组的最大乘积。在计算 `DP(i+1)`的时候，有个小问题： 如果 `A[i] > 0, DP(i) > 0` 是正的，继续乘可以增大最大值； 但如果 `DP(i) < 0`，继续乘会减小值：还不如 `A[i]`本身； 如果`A[i] < 0`，则需要最负的 `DP(i)`，而不是最正的，这样才能得到最大值。

所以，`DP(i)` 需要记录两个值： 最大值和最小值（pmax和pmin， `pmax >= pmin`）。 遇到正数： `A[i] * pmax > A[i] * pmin`，遇到负数： `A[i] * pmax < A[i] * pmin`。 另外，还要将结果与 `A[i]`比较，如果都不如 `A[i]`，还要用`A[i]`替换掉。

代码如下：


In [155]:
class Solution:
    def maxProduct(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        result = pmax = pmin = p = nums[0]
        for n in nums[1:]:
            if n < 0:
                pmax, pmin = pmin, pmax
            pmax = max(n, n * pmax)
            pmin = min(n, n * pmin)
            result = max(result, pmax)
        return result

Solution().maxProduct([-2])
            

-2

如果不考虑正负号什么乱七八糟的东西，简单粗暴

会稍微慢一点点。

In [156]:
class Solution:
    def maxProduct(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        result = pmax = pmin = p = nums[0]
        for n in nums[1:]:
            r = (n, n * pmin, n * pmax)
            pmax = max(r)
            pmin = min(r)
            result = max(result, pmax)
        return result

## ☆☆198. House Robber

You are a professional robber planning to rob houses along a street. Each house has a certain amount of money stashed, the only constraint stopping you from robbing each of them is that adjacent houses have security system connected and it will automatically contact the police if two adjacent houses were broken into on the same night.

Given a list of non-negative integers representing the amount of money of each house, determine the maximum amount of money you can rob tonight without alerting the police.

题目大意是： 有一排商店，里面有现金（非负）。唯一的限制是如果抢了连续两个商店，会自动报警。 问：如果计划抢劫，获取最多的现金？

一开始我以为跟 Maximum Subarray 一样，求最大和的问题。后来想，这也太简单了，奇偶分别求和，取最大值不就完了？ 后来发现，事情没那么简单。比如 `[3, 2, 1, 2]`， 这样做的结果是 4，实际上，取头尾两个值，是5。

其实本质上仍然是DP问题。 也很简单，限制相邻条件就可以了。

定义 `DP(i)` 是问题 `A[:i]`的解。 那么求 `DP(i+1)` 的时候， 看 `A[i]`，这里包括两个解： 包括 `A[i]`的解，和不包括 `A[i]`的解。 包括 `A[i]`的解，要求 `DP(i)`的不包括 `A[i-1]`的解； 不包括 `A[i]`的解， 则 等于 `DP(i)`。

所以 `DP(i)` 需要记录两个解，包括 `A[i-1]`的解，和不包括 `A[i-1]`的解。最终取一个最大值即可。

- `DP(i+1, inclue A[i]) = max(DP(i, not inclue A[i-1]) + A[i], DP(i, inclue A[i-1])`
- `DP(i+1, not include A[i]) =  DP(i, inclue A[i-1])`



In [162]:
class Solution:
    def rob(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        a = b = 0
        for n in nums:
            a, b = max(b + n, a), a
        return max(a, b)
    
Solution().rob([3, 2, 1, 2])

5

## ☆☆213. House Robber II

Note: This is an extension of House Robber.

After robbing those houses on that street, the thief has found himself a new place for his thievery so that he will not get too much attention. This time, all houses at this place are arranged in a circle. That means the first house is the neighbor of the last one. Meanwhile, the security system for these houses remain the same as for those in the previous street.

Given a list of non-negative integers representing the amount of money of each house, determine the maximum amount of money you can rob tonight without alerting the police.

上一个问题的扩展。 现在不是一排房子了，而是首尾连成环。。仍然满足抢劫相邻房间就会报警的条件。


乍一看很棘手。其实解决问题思路比较巧妙（看讨论区的）

`A[0]` 和 `A[-1]` 肯定不能同时出现在最优解里了。 所以问题可以变成： `A[:-1]` 和 `A[1:]`的问题。 结果是二者的最大值。仅此而已。

In [165]:
class Solution:
    def rob(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        if len(nums) == 1:
            return nums[0]
        def _rob(A):
            a = b = 0
            for n in A:
                a, b = max(b + n, a), a
            return max(a, b)
        
        return max(_rob(nums[1:]), _rob(nums[:-1]))
    
Solution().rob([3, 2, 1, 2])

4

## 279. Perfect Squares

Given a positive integer n, find the least number of perfect square numbers (for example, 1, 4, 9, 16, ...) which sum to n.

For example, given n = 12, return 3 because 12 = 4 + 4 + 4; given n = 13, return 2 because 13 = 4 + 9.


如果定义成DP问题，实际上也很简单 `DP(n) = min(DP(i) + DP(n-i))`， 其中 $i \in [1, n // 2]$

再仔细想一下，可以更简化： 因为 `n` 的分解至少包括一个完全平方数，所以 `DP(n) = min(DP(n-i*i)) + 1`， 其中 $i \in [1, sqrt(n)]$

另外对于 完全平方数， `DP(n) = 1`

这样的计算复杂度是 $O(n \sqrt{n})$，其实挺差的

这个问题有很多解法，参考 <https://leetcode.com/problems/perfect-squares/discuss/71488/Summary-of-4-different-solutions-(BFS-DP-static-DP-and-mathematics)> 甚至还可以数值求解



In [180]:
## 超时了。

class Solution:
    def numSquares(self, n):
        """
        :type n: int
        :rtype: int
        """
        dp = [0] * (n + 1)
        for i in range(1, n+1):
            dp[i] = dp[i-1] + 1
            j = 2
            while j * j <= i:
                dp[i] = min(dp[i - j*j] + 1, dp[i])
                j += 1  
        return dp[n]

Solution().numSquares(12)
Solution().numSquares(5)

2

In [188]:
# 使用缓存，哈哈

class Solution:
    _dp = [0, 1, 2, 3, 1]
    def numSquares(self, n):
        """
        :type n: int
        :rtype: int
        """
        dp = self._dp
        while len(dp) <= n:
            dp.append(min(dp[-i*i] for i in range(1, int(len(dp)**0.5+1))) + 1)
        return dp[n]
    

Solution().numSquares(12)
Solution().numSquares(5)   

2

In [185]:
# 数直解法，没看懂 


class Solution:
    def numSquares(self, n):
        """
        :type n: int
        :rtype: int
        """
        import math
        def is_square(_n):
            sqrt = int(math.sqrt(_n))
            return sqrt * sqrt == _n
        
        # 1
        if is_square(n):
            return 1
        # 4
        while n & 3 == 0:
            n = n >> 2
        if n & 7 == 7:
            return 4
        
        # 2
        sqrt_n = int(math.sqrt(n))
        for i in range(sqrt_n + 1):
            if (is_square(n - i * i)):
                return 2
        
        return 3;        

Solution().numSquares(12)
Solution().numSquares(5)

2

In [184]:
5 & 3

1

## 300. Longest Increasing Subsequence

最长递增序列

给定一个数组序列， 找出其中最长的单调递增子序列

Given an unsorted array of integers, find the length of longest increasing subsequence.

For example,
Given [10, 9, 2, 5, 3, 7, 101, 18],
The longest increasing subsequence is [2, 3, 7, 101], therefore the length is 4. Note that there may be more than one LIS combination, it is only necessary for you to return the length.

Your algorithm should run in O(n2) complexity.

Follow up: Could you improve it to O(n log n) time complexity?


也是典型的 DP问题。不过稍微复杂一点。

定义 `DP(i)` 为  `A[:i+1]` 问题的解。则 `DP(i)`默认为1，看 `A[i]`，如果 `A[i]` 比 `A[j]` 大， 则 `DP(i) = DP(j) + 1`，其中 `j < i`。 所以 `DP(i) = max(DP(j) + 1  for j in range(i)`

这种方法的计算时间为 $O(n^2)$

In [213]:
class Solution:
    def lengthOfLIS(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        if len(nums) == 0:
            return 0
        dp = [0] * len(nums)
        for i in range(len(nums)):
            dp[i] = 1
            for j in range(i):
                if nums[i] > nums[j]:
                    dp[i] = max(dp[j] + 1, dp[i])
            # print(dp)
        return max(dp)

# Solution().lengthOfLIS([10, 9, 2, 5, 3, 7, 101, 18])
Solution().lengthOfLIS([1,3,6,7,9,4,10,5,6])

6


有 $O(n \log n)$ 的解法， DP问题的定义并不一样，而且很难想。。。并且很难定义。

定义 `DP(i)` 为 `A[:i+1]` 数组中的元素组成的一个递增的序列。这个序列的长度是问题的最优解（但是本身可能并不是符合条件的递增子序列）。

对于 `A[i]`，如果 `A[i] >= DP(i-1)[-1]`，则 `DP(i) = DP(i-1) append A[i]`

否则，用二分查找法，在 `DP(i-1)` 中找到位置 j， 使得 `DP(i-1)[j-1] <= A[i] < DP(i-1)[j]`，将 `DP(i-1)[j]` 替换成 `A[i]` 即可。


In [220]:
class Solution:
    def lengthOfLIS(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        if len(nums) == 0:
            return 0
        dp = [nums[0]]
        # print(nums)
        for n in nums[1:]:
            i, j = 0, len(dp)
            while i < j:
                c = (i + j) // 2
                if n < dp[c]:
                    j = c
                else:
                    i = c + 1
            if i == len(dp):
                dp.append(n)
            else:
                dp[i] = n                
            # print(dp)
        return len(dp)
    
# Solution().lengthOfLIS([1,3,6,7,9,4,10,5,6])
Solution().lengthOfLIS([2, 2])

2

In [ ]:

上面是 递增。如果要求严格递增（题目要求），则有些改变

定义 `DP(i)` 为 `A[:i+1]` 数组中的元素组成的一个严格递增的序列。这个序列的长度是问题的最优解（但是本身可能并不是符合条件的递增子序列）。


用二分查找法，在 `DP(i-1)` 中找到位置 j， 使得 `DP(i-1)[j-1] < A[i] <= DP(i-1)[j]`，将 `DP(i-1)[j]` 替换成 `A[i]` 即可。

In [222]:
class Solution:
    def lengthOfLIS(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        if len(nums) == 0:
            return 0
        dp = [nums[0]]
        # print(nums)
        for n in nums[1:]:
            i, j = 0, len(dp)
            while i < j:
                c = (i + j) // 2
                if n <= dp[c]:
                    j = c
                else:
                    i = c + 1
            if i == len(dp):
                dp.append(n)
            else:
                dp[i] = n                
            # print(dp)
        return len(dp)
    
Solution().lengthOfLIS([1,3,6,7,9,4,10,5,6])
# Solution().lengthOfLIS([2, 2])

6

## 303. Range Sum Query - Immutable

Given an integer array nums, find the sum of the elements between indices i and j (i ≤ j), inclusive.

```
Example:
Given nums = [-2, 0, 3, -5, 2, -1]

sumRange(0, 2) -> 1
sumRange(2, 5) -> -1
sumRange(0, 5) -> -3
```

超简单的问题。但是不允许 `sum(nums[i:j+1])` 这种做法。目的是 $O(1)$的方法获得 `sumRange(i,j)`

允许使用缓存，空间复杂度控制在 $O(n)$。

觉得吧，算是空间上的 DP问题吧。。。。其实也是很简单，计算所有累积和 `csum[i] = sum(nums[:i])`， `sumRange(i,j) = csum[j+1] - csum[i]`

In [223]:
class NumArray:

    def __init__(self, nums):
        """
        :type nums: List[int]
        """
        csum = [0]
        for n in nums:
            csum.append(csum[-1] + n)
        self.csum = csum

    def sumRange(self, i, j):
        """
        :type i: int
        :type j: int
        :rtype: int
        """
        return self.csum[j+1] - self.csum[i]

obj = NumArray([-2, 0, 3, -5, 2, -1])
obj.sumRange(2,5)

-1

## 304. Range Sum Query 2D - Immutable

Given a 2D matrix matrix, find the sum of the elements inside the rectangle defined by its upper left corner (row1, col1) and lower right corner (row2, col2).

就是上一个问题变成2D的。

思路同理。令 `csum[i,j] = sum(matrix[:i, :j])`, 则 `sumRegion(row1, col1, row2, col2) = csum[row2+1, col2+1] - csum[row1, col2+1] - csum[row2+1, col1] + csum[row1, col1]`，仅此而已。

In [228]:
class NumMatrix:

    def __init__(self, matrix):
        """
        :type matrix: List[List[int]]
        """
        rows = len(matrix)
        if rows == 0:
            return
        cols = len(matrix[0])
        csum = [[0] * (cols + 1) for _ in range(rows + 1)]
        for i in range(rows):
            for j in range(cols):
                csum[i+1][j+1] = matrix[i][j] + csum[i][j+1] + csum[i+1][j] - csum[i][j]
        self.csum = csum
        
    def sumRegion(self, row1, col1, row2, col2):
        """
        :type row1: int
        :type col1: int
        :type row2: int
        :type col2: int
        :rtype: int
        """
        s = self.csum
        return s[row2+1][col2+1] - s[row2+1][col1] -s[row1][col2+1] + s[row1][col1]
    

# Your NumMatrix object will be instantiated and called as such:
obj = NumMatrix([[3,0,1,4,2],[5,6,3,2,1],[1,2,0,1,5],[4,1,0,1,7],[1,0,3,0,5]])
print(obj.sumRegion(2,1,4,3))

8


## 121. Best Time to Buy and Sell Stock

Say you have an array for which the ith element is the price of a given stock on day i.

If you were only permitted to complete at most one transaction (ie, buy one and sell one share of the stock), design an algorithm to find the maximum profit.

Example 1:
Input: [7, 1, 5, 3, 6, 4]
Output: 5

max. difference = 6-1 = 5 (not 7-1 = 6, as selling price needs to be larger than buying price)
Example 2:
Input: [7, 6, 4, 3, 1]
Output: 0

In this case, no transaction is done, i.e. max profit = 0.

先做了 309题目之后，再做这道题目就很简单。 但是也有些区别，不可完全照搬。这个题目要求最多只有一次 buy、sell操作。

`DP(i)` 是第 i天的操作之后的最优解，分为两个状态： buy之后的和sell之后的。要求： 在最低时买入，最高时卖出。

- `DP(i, buy) = min(prices[i], DP(i-1, buy)`
- `DP(i, sell) = max(DP(i-1, sell), prices[i] - DP(i-1, buy))`

In [285]:

class Solution:
    def maxProfit(self, prices):
        """
        :type prices: List[int]
        :rtype: int
        """
        if not prices:
            return 0
        
        buy, sell = prices[0], 0
        
        # print(buy, sell, rest)
        for p in prices:
            buy, sell = min(p, buy), max(sell, p - buy)
            # print(p, buy, sell)
        return sell
    
Solution().maxProfit([7, 1, 5, 3, 6, 4])
# Solution().maxProfit([2, 1])

5

## ☆☆ 309. Best Time to Buy and Sell Stock with Cooldown

Say you have an array for which the ith element is the price of a given stock on day i.

Design an algorithm to find the maximum profit. You may complete as many transactions as you like (ie, buy one and sell one share of the stock multiple times) with the following restrictions:

You may not engage in multiple transactions at the same time (ie, you must sell the stock before you buy again).
After you sell your stock, you cannot buy stock on next day. (ie, cooldown 1 day)
Example:

prices = [1, 2, 3, 0, 2]
maxProfit = 3
transactions = [buy, sell, cooldown, buy, sell]

最优解问题。

题意是： 数列显示一只股票每天的价格。 一天只能做一个操作 buy 或者 sell，但是sell之后有一个冷却时间 1天。 问如何操作，获得最大收益。

题意补充： sell之前必须先buy； sell之后必须歇一天； buy之后不可连续buy，必须sell，但中间可以歇几天； 

这个DP问题， `DP(i)` 表示第 `i` 天结束后的最大收益，但是有三种状态： buy之后的； sell之后的； cooldown 之后的。注意这是一个状态机。 buy状态和cooldown状态可以rest回到自身； cooldown状态经过buy动作到达buy状态； buy状态经过sell动作进入 sell状态； sell状态跳转到 cooldown状态。

状态机模型参考 <https://leetcode.com/problems/best-time-to-buy-and-sell-stock-with-cooldown/discuss/75928/Share-my-DP-solution-(By-State-Machine-Thinking)>  但这个问题本质上是一个DP问题。 状态机模型可以帮助更好理清思路，明确最大收益的路径来源，以及各变量的意义。

- buy之后的状态： `DP(i, buy) = max(DP(i-1, rest) - prices[i], DP(i-1, buy)`，最大收益，来源，两个。 
- sell之后的状态： `DP(i, sell) = DP(i-1, buy) + prices[i]`，这个状态来源只有一个。
- cooldown之后的状态： `DP(i, rest) = max(DP(i-1, sell), DP(i-1, rest))`

最后再说一遍， 这个DP问题，隐含着路径规划，状态转移。类似于 #198 #53 #152 这几个问题。

In [245]:
class Solution:
    def maxProfit(self, prices):
        """
        :type prices: List[int]
        :rtype: int
        """
        if not prices:
            return 0
        
        buy, sell, rest = -prices[0], 0, 0
        
        # print(buy, sell, rest)
        for p in prices:
            buy, sell, rest = max(rest - p, buy), buy + p, max(sell, rest)
            # print(p, buy, sell, rest)
        return max(buy, sell, rest)
    
Solution().maxProfit([1, 2, 3, 0, 2])
Solution().maxProfit([2, 1])

0

一个小教训： `max(1, 2, 3)` 速度要比 `max((1, 2, 3))` 要快。后者用到一个迭代对象。

##  70. Climbing Stairs

You are climbing a stair case. It takes n steps to reach to the top.

Each time you can either climb 1 or 2 steps. In how many distinct ways can you climb to the top?

Note: Given n will be a positive integer.

不能再简单的DP问题。实际上是一个递归问题，不是最优解问题。

- `DP(n)` 表示 爬 n 台阶的解。
- `DP(n) = DP(n-1) + DP(n-2)`

其实就是一个斐波那契数列。 上面非递归解法是 $O(n)$复杂度。还有更变态的 $O(\log n)$ 解法，用到 矩阵乘法。参考 <https://leetcode.com/problems/climbing-stairs/solution/>

In [246]:
class Solution:
    def climbStairs(self, n):
        """
        :type n: int
        :rtype: int
        """
        p1, p2 = 0, 1
        for i in range(n):
            p1, p2 = p2, p1 + p2
        
        return p2
    
Solution().climbStairs(2)

2

## 321. Create Maximum Number



## 322. Coin Change

You are given coins of different denominations and a total amount of money amount. Write a function to compute the fewest number of coins that you need to make up that amount. If that amount of money cannot be made up by any combination of the coins, return -1.

Example 1:
coins = [1, 2, 5], amount = 11
return 3 (11 = 5 + 5 + 1)

Example 2:
coins = [2], amount = 3
return -1.

Note:
You may assume that you have an infinite number of each kind of coin.

呵呵，硬币问题。 以前难死我了。 DP问题。

比如对于 `[1, 2, 5]`, `DP(n) = min(DP(n-5), DP(n-2), DP(n-1)) + 1`

In [252]:
# 递归的方式超时了，用非递归方式

class Solution:
    def coinChange(self, coins, amount):
        """
        :type coins: List[int]
        :type amount: int
        :rtype: int
        """
        if amount == 0:
            return 0
        
        INF = 2 ** 30
        def solve(n):
            if n < 0:
                return INF
            if n == 0:
                return 0
            return min((solve(n-i) for i in coins)) + 1
        
        result = solve(amount)
        return -1 if result > INF  else result

Solution().coinChange([1, 2, 5], 11)

3

In [259]:
# 这个过了

class Solution:
    def coinChange(self, coins, amount):
        """
        :type coins: List[int]
        :type amount: int
        :rtype: int
        """
        if amount == 0:
            return 0
        
        INF = 2 ** 30
        dp = [INF] * (amount + 1)
        dp[0] = 0
        
        for n in range(1, amount + 1): 
            for coin in coins:
                if n - coin >= 0:
                    dp[n] = min(dp[n-coin] + 1, dp[n])
                    
        return dp[-1] if dp[-1] < INF else -1

Solution().coinChange([1, 2, 5], 11)
Solution().coinChange([2], 3)

3

感觉用 DFS，并且优先用大硬币，要快一些。并且采取 剪枝策略。

例如：
```
11  --(-5*q?)                         #1 看剩余的能不能整除此面值的coin，如果能，就不用往下搜索。
    --(-5) -- 6 --(-5) -- 1 -- (-2?)  #2, 3, 4
                            -- (-1?)  #5  solution = 3
                --(-2*q?) -- q = 3    #6  count + q = 4 >  solution    隐藏着剪枝III， 停止在6上的搜索。
                --(-1*q?) -- q = 6    # 在 6 这个分支上， 因为全部用2的解都不是最优解； 那么全部用1的解肯定也不是最优解。
    --(-2*q?) q = 5                   #7  count + q + 1 = 6 > solution  剪枝III
    --(-1*q?), q = 11                 # 在 11 这个分支上，因为全部用2都不是最优解，那么全部用1也不是最优解
    
``` 
注意这里用了coin=2的硬币，就可以不用考虑coin=5的硬币，所以是降序排列coins。 每个分支都是一种硬币组合，只需要考虑降序顺序就可以了。

剪枝策略有：

1. 如果 count >= solution，终止分支搜索
2. 如果 剩余部分可以被coin整除，计算 count + q，取  min(count+q, solution)，终止分支搜索
2. 如果 剩余部分不可被coin整除，并且 count + q + 1 > solution， 则终止分支搜索。

以上剪枝策略非常有效的减少了计算时间。

In [275]:
class Solution:
                
    def coinChange(self, coins, amount):
        """
        :type coins: List[int]
        :type amount: int
        :rtype: int
        """
        if amount == 0:
            return 0
        
        coins.sort()
        solution = amount + 1
        
        def dfs(idx, remain, count, checked=False):
            nonlocal solution            
            # print(solution, remain, coins[idx], count)
            # cutI
            if count >= solution:
                return

            if not checked:
                q = remain // coins[idx]
                
                # cut III
                if remain % coins[idx] == 0:
                    solution = min(solution, count + q)
                    return
                # cut III
                elif count + q + 1 >= solution:
                    return

            if idx > 0:
                if coins[idx] < remain:
                    dfs(idx, remain - coins[idx], count + 1, True)
                dfs(idx-1, remain, count)

        dfs(len(coins) - 1, amount, 0)

        return -1 if solution > amount else solution
        

Solution().coinChange([1, 2, 5], 11)

12 11 5 0
12 6 5 1
12 1 5 2
12 1 2 2
12 1 1 2
3 6 2 1
3 11 2 0


3

## 813. Largest Sum of Averages

We partition a row of numbers A into at most K adjacent (non-empty) groups, then our score is the sum of the average of each group. What is the largest score we can achieve?

Note that our partition must use every number in A, and that scores are not necessarily integers.

Example:
Input: 
A = [9,1,2,3,9]
K = 3
Output: 20
Explanation: 
The best choice is to partition A into [9], [1, 2, 3], [9]. The answer is 9 + (1 + 2 + 3) / 3 + 9 = 20.
We could have also partitioned A into [9, 1], [2], [3, 9], for example.
That partition would lead to a score of 5 + 2 + 6 = 13, which is worse.


题目大意是给定 一个数列 A， 按顺序划分成相邻的K组，每组不为空。 找出每组平均数 之和的最大值。

动态规划问题。 定义 `DP(i, k)` 为 问题 `A[i:]` 划分成 k 组的最优解。

- `DP(i, 1) = average(A[i:]) `
- `DP(i, k) = min( average(A[i:j]) + DP(j, k-1))`。 

In [286]:
class Solution:
    def largestSumOfAverages(self, A, K):
        """
        :type A: List[int]
        :type K: int
        :rtype: float
        """
        
        P = [0]
        for x in A: 
            P.append(P[-1] + x)
            
        def average(i, j):
            return (P[j] - P[i]) / float(j - i)

        N = len(A)
        dp = [average(i, N) for i in range(N)]
        for k in range(1, K):
            for i in range(N):
                for j in range(i+1, N):
                    dp[i] = max(dp[i], average(i, j) + dp[j])
        return dp[0]
    
Solution().largestSumOfAverages([9,1,2,3,9], 3)

20.0

## 808. Soup Servings

There are two types of soup: type A and type B. Initially we have N ml of each type of soup. There are four kinds of operations:

Serve 100 ml of soup A and 0 ml of soup B
Serve 75 ml of soup A and 25 ml of soup B
Serve 50 ml of soup A and 50 ml of soup B
Serve 25 ml of soup A and 75 ml of soup B
When we serve some soup, we give it to someone and we no longer have it.  Each turn, we will choose from the four operations with equal probability 0.25. If the remaining volume of soup is not enough to complete the operation, we will serve as much as we can.  We stop once we no longer have some quantity of both types of soup.

Note that we do not have the operation where all 100 ml's of soup B are used first.  

Return the probability that soup A will be empty first, plus half the probability that A and B become empty at the same time.


注意这里可以简化数字： 令`N = ceil(N / 25)`， 以25ml为一份这样的单位计算。 `DP(a, b)` 为 剩余 a份A, b份 B 的解。则：

- `DP(a, b) = 0.25 * (DP(a-4, b) + DP(a-3, b-1) + DP(a-2, b-2) + DP(a-3, b-1))` 
- `DP(a <=0, b<=0) = 0.5`
- `DP(a <=0, b>0) = 1`
- `DP(a > 0, b<=0) = 0`




In [305]:
class Solution(object):
    _cache = {}
    def soupServings(self, N):
        if N > 5575:
            return 1
        n = math.ceil(N / 25)
        cache = self._cache
        def dp(a, b):
            if (a, b) not in cache:
                if a <= 0:
                    ans = 1.0 if b > 0 else 0.5
                elif b <= 0:
                    ans = 0.0
                else:
                    ans = 0.25 * (dp(a-4, b) + dp(a-3, b-1) + dp(a-2, b-2) + dp(a-1, b-3))
                cache[a, b] = ans
            return cache[a, b]

        return dp(n, n)
    
for n in range(500, 10000, 25):
    ans = Solution().soupServings(n)
    if 1 - ans < 1e-6:
        print(n)
        break

5575


In [313]:
# 最快的答案之一。其实关键是将 N 从 5575 改成了4800，少算了很多。

class Solution(object):
    _cache = {}
    def soupServings(self, N):
        if N > 5575:
            return 1
        
        n = math.ceil(N / 25)
        cache = self._cache
        def dp(a, b):
            if (a, b) not in cache:
                if a <= 0:
                    ans = 1.0 if b > 0 else 0.5
                elif b <= 0:
                    ans = 0.0
                else:
                    ans = 0.25 * (dp(a-4, b) + dp(a-3, b-1) + dp(a-2, b-2) + dp(a-1, b-3))
                cache[(a, b)] = ans
            return cache[(a, b)]

        return dp(n, n)
    
# for n in range(0, 5600, 25):
#     print(Solution().soupServings(n), end=',\n')

In [314]:
# 最无耻的方法，但是是最快的，哈哈

answer = [
    0.5,
    0.625,
    0.625,
    0.65625,
    0.71875,
    0.7421875,
    0.7578125,
    0.78515625,
    0.796875,
    0.81787109375,
    0.82763671875,
    0.8448486328125,
    0.8521728515625,
    0.86669921875,
    0.87255859375,
    0.88482666015625,
    0.8896331787109375,
    0.9000759124755859,
    0.9040584564208984,
    0.9130053520202637,
    0.916344165802002,
    0.9240454435348511,
    0.9268695116043091,
    0.9335255026817322,
    0.9359304308891296,
    0.9417028725147247,
    0.9437624514102936,
    0.9487833306193352,
    0.9505554363131523,
    0.9549337066709995,
    0.9564644806087017,
    0.960290864109993,
    0.961617625085637,
    0.9649682220479008,
    0.9661214997468051,
    0.9690605557043455,
    0.9700655688720872,
    0.9726476206215011,
    0.9735253741437191,
    0.9757969387565026,
    0.9765650521094358,
    0.9785659726134099,
    0.979239316208691,
    0.9810038450348486,
    0.9815950361329726,
    0.9831527156789832,
    0.983672509785066,
    0.985048894078048,
    0.9855064973468473,
    0.986723747872853,
    0.9871270689654299,
    0.9882044498665583,
    0.988560304803165,
    0.9895145968775682,
    0.9898288769838451,
    0.9906747256420321,
    0.9909525362169398,
    0.9917027453281975,
    0.991948520868163,
    0.9926143019817137,
    0.9928319024738018,
    0.9934230870161792,
    0.993615878826885,
    0.9941410994147383,
    0.9943120235564422,
    0.9947788694180789,
    0.9949304994069486,
    0.9953456500002973,
    0.9954802410813399,
    0.995849581274638,
    0.9959691124487241,
    0.9962978320513973,
    0.9964040420714921,
    0.9966967220350943,
    0.9967911403272177,
    0.9970518275561406,
    0.9971358008517697,
    0.9973680732525854,
    0.9974427885837021,
    0.9976498117264034,
    0.9977163163248763,
    0.9979008928780695,
    0.9979601114320225,
    0.9981247243587498,
    0.9981774740057776,
    0.9983243243603961,
    0.9983713277376419,
    0.9985023677817058,
    0.998544264403388,
    0.9986612266555319,
    0.9986985828474888,
    0.9988030055954544,
    0.9988363231820787,
    0.9989295729115085,
    0.9989592968218388,
    0.9990425879540417,
    0.9990691128897385,
    0.9991435251674701,
    0.999167201455148,
    0.9992336952700617,
    0.99925483400331,
    0.9993142639198735,
    0.9993331414817622,
    0.9993862681790912,
    0.9994031302235895,
    0.9994506310479657,
    0.9994656960081456,
    0.9995081743042766,
    0.999521636486147,
    0.9995596298538699,
    0.9995716621669473,
    0.9996056497716175,
    0.999616406140688,
    0.9996468151894824,
    0.9996564326862809,
    0.9996836441687537,
    0.9996922448973439,
    0.9997165986764868,
    0.999724291441854,
    0.999746090771399,
    0.9997529725570642,
    0.9997724880915861,
    0.9997786453688446,
    0.9997961187252201,
    0.9998016286138132,
    0.9998172755355883,
    0.999822206833195,
    0.9998362200032871,
    0.9998406340991068,
    0.9998531856409006,
    0.9998571373267513,
    0.9998683810289422,
    0.9998719192196881,
    0.9998819925160916,
    0.9998851608898072,
    0.9998941866217235,
    0.9998970241887645,
    0.9999051121742936,
    0.9999076537833625,
    0.9999149022152549,
    0.9999171790036043,
    0.9999236756947564,
    0.9999257154888351,
    0.9999315389823528,
    0.9999333666544716,
    0.999938587213306,
    0.9999402249992518,
    0.9999449054887031,
    0.9999463732706648,
    0.9999505699455563,
    0.9999518855042262,
    0.9999556487112167,
    0.9999568279504891,
    0.9999602027548221,
    0.9999612599021226,
    0.9999642866470688,
    0.9999652344320045,
    0.9999679492383364,
    0.9999687990520527,
    0.9999712342640741,
    0.9999719963014364,
    0.9999741808853659,
    0.9999748642718481,
    0.9999768241717177,
    0.9999774370766661,
    0.999979195532326,
    0.9999797452700848,
    0.9999813231014011,
    0.9999818162216201,
    0.9999832320825048,
    0.9999836744508008,
    0.9999849450563164,
    0.999985341926334,
    0.9999864822557589,
    0.9999868383335604,
    0.9999878618119905,
    0.9999881813136011,
    0.9999890999743808,
    0.9999893866772255,
    0.9999902113072541,
    0.999990468596144,
    0.9999912088658822,
    0.9999914397741335,
    0.9999921043539375,
    0.9999923116001441,
    0.9999929082643784,
    0.9999930942853033,
    0.9999936300055312,
    0.9999937969855307,
    0.9999942780139369,
    0.9999944279112863,
    0.9999948598553648,
    0.999994994425816,
    0.9999953823152488,
    0.9999955031331131,
    0.9999958514796593,
    0.9999959599566788,
    0.999996272807815,
    0.9999963702100552,
    0.9999966511970214,
    0.9999967386599964,
    0.9999969910408368,
    0.999997069583052,
    0.9999972962811772,
    0.999997366816257,
    0.9999975704549949,
    0.9999976338025459,
    0.9999978167361046,
    0.9999978736314437,
    0.9999980379726587,
    0.9999980890755304,
    0.9999982367207354,
    0.99999828262312,
    0.9999984152744386,
    0.9999984565075495,
    0.9999985756928802,
    0.9999986127334335,
    0.9999987198243664,
    0.9999987531001999,
    0.9999988493280795,
    0.9999988792231894,
    0.9999989656935195,
    0.9999989925525751,
    0.999999070257936,
]

class Solution:
    def soupServings(self, N):
        """
        :type N: int
        :rtype: float
        """
        if N > 5575:
            return 1
        n = math.ceil(N / 25)
        return answer[n]
    
print(Solution().soupServings(1000))

0.9765650521094358
